In [5]:
# package 导入
import yaml
import cv2

import mindspore
from mindspore import nn, ops

from common import autopad

In [ ]:
# model构建
class Conv(nn.Cell):
    """Standard convolution"""
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, act=True):
        """
        :param c1: channel input
        :param c2: channel output
        :param k: kernel
        :param s: stride
        :param p: padding
        :param g: groups
        """
        super().__init__()
        # 默认通道顺序：NCHW
        self.conv = nn.Conv2d(in_channels=c1, out_channels=c2, kernel_size=k, stride=s, pad_mode='pad', padding=autopad(k, p), group=g, has_bias=False)
        self.bn = nn.BatchNorm2d(num_features=c2, eps=1e-5, momentum=0.1)
        self.act = 


class Model(nn.Cell):
    """定义yolov5模型"""
    def __init__(self, cfg='yolov5s.yaml', ch=3, nc=None, anchors=None):
        """
        :param cfg: 模型初始化参数
        :param ch: input channels
        :param nc: number of classes
        :param anchors: anchor 参数
        """
        super().__init__()
        with open(cfg, encoding='ascii', errors='ignore') as f:
            self.yaml = yaml.safe_load(f)  # model dict
            
        # define model
        ch = self.yaml['ch'] = self.yaml.get('ch', ch)  # input channels
        if nc and nc!=self.yaml['nc']:
            print("overriding model.yaml nc!!!")
            self.yaml['nc'] = nc
        if anchors:
            print("overriding model.yaml anchors!!!")
            self.yaml['anchors'] = round(anchors)   # 四舍五入？
        
        self.anchors, self.nc, self.gd, self.gw = \
            self.yaml['anchors'], self.yaml['nc'], self.yaml['depth_multiple'], self.yaml['width_multiple']
        
        na = (len(anchors[0]) // 2) if isinstance(anchors, list) else anchors   # number of anchors per layer
        no = na * (nc + 5)   # numer of outputs   classes + pos + conf
        
        